## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-20-54-00 +0000,wsj,Senators Seek Last-Minute Deal on Immigration ...,https://www.wsj.com/politics/policy/senators-s...
1,2026-01-27-20-49-48 +0000,nyt,Trump Vows Higher Tariffs for South Korea Mont...,https://www.nytimes.com/2026/01/27/world/asia/...
2,2026-01-27-20-49-23 +0000,nyt,Spain Offers Undocumented Migrants a Legal Way...,https://www.nytimes.com/2026/01/27/world/europ...
3,2026-01-27-20-47-31 +0000,nyt,Texas Moves to Curtail Visas for Skilled Forei...,https://www.nytimes.com/2026/01/27/us/texas-im...
4,2026-01-27-20-46-00 +0000,wsj,Treasury Yields Mixed Ahead of Expected Fed Hold,https://www.wsj.com/finance/short-dated-jgbs-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2442/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,65
48,ice,26
43,minneapolis,21
78,minnesota,19
238,year,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2026-01-27-20-37-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,154
92,2026-01-27-18-14-21 +0000,wapo,Trump sends border czar to Minneapolis as Alex...,https://www.washingtonpost.com/nation/2026/01/...,143
297,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...,141
247,2026-01-27-10-28-15 +0000,cbc,Judge in Minnesota orders acting ICE director ...,https://www.cbc.ca/news/world/bovino-ice-minne...,141
189,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...,140


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,154,2026-01-27-20-37-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
117,104,2026-01-27-17-42-16 +0000,nypost,Alex Pretti’s sister slams ‘disgusting lies’ a...,https://nypost.com/2026/01/27/us-news/alex-pre...
309,65,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
155,63,2026-01-27-15-59-40 +0000,nypost,Meta and YouTube face landmark trial over yout...,https://nypost.com/2026/01/27/business/meta-ti...
203,59,2026-01-27-13-22-00 +0000,wsj,EU and India Reach Free-Trade Deal as World Re...,https://www.wsj.com/economy/trade/eu-and-india...
281,56,2026-01-27-03-46-00 +0000,wsj,Democrats want changes to the Department of Ho...,https://www.wsj.com/politics/policy/government...
223,46,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...
87,43,2026-01-27-18-23-00 +0000,wsj,Amazon to Shut Down All Amazon Go and Amazon F...,https://www.wsj.com/business/retail/amazon-to-...
73,42,2026-01-27-18-47-28 +0000,cbc,"At least 35 dead after winter storm hits U.S.,...",https://www.cbc.ca/news/world/southern-states-...
21,40,2026-01-27-20-16-27 +0000,nypost,"Border Patrol agent shoots, wounds man near US...",https://nypost.com/2026/01/27/us-news/border-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
